In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
#     eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
#                                                                      labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:0.7291 eloss:2.4625 aloss2:1.3461 exploreP:0.9986
Episode:1 meanR:21.5000 R:29.0000 rate:0.0580 aloss:0.7250 eloss:2.4825 aloss2:1.3327 exploreP:0.9958
Episode:2 meanR:19.3333 R:15.0000 rate:0.0300 aloss:0.7236 eloss:2.4452 aloss2:1.3372 exploreP:0.9943
Episode:3 meanR:21.2500 R:27.0000 rate:0.0540 aloss:0.7037 eloss:2.5052 aloss2:1.3106 exploreP:0.9916
Episode:4 meanR:23.0000 R:30.0000 rate:0.0600 aloss:0.7174 eloss:2.4462 aloss2:1.3211 exploreP:0.9887
Episode:5 meanR:24.6667 R:33.0000 rate:0.0660 aloss:0.7107 eloss:2.4544 aloss2:1.3120 exploreP:0.9855
Episode:6 meanR:23.0000 R:13.0000 rate:0.0260 aloss:0.7109 eloss:2.4418 aloss2:1.3110 exploreP:0.9842
Episode:7 meanR:23.1250 R:24.0000 rate:0.0480 aloss:0.7079 eloss:2.4537 aloss2:1.2988 exploreP:0.9819
Episode:8 meanR:22.0000 R:13.0000 rate:0.0260 aloss:0.7066 eloss:2.4500 aloss2:1.3004 exploreP:0.9806
Episode:9 meanR:21.8000 R:20.0000 rate:0.0400 aloss:0.7022 eloss:2.4887 aloss2:1.2

Episode:80 meanR:28.2222 R:11.0000 rate:0.0220 aloss:0.7055 eloss:1.9895 aloss2:1.4551 exploreP:0.7977
Episode:81 meanR:28.0000 R:10.0000 rate:0.0200 aloss:0.7202 eloss:1.9734 aloss2:1.4579 exploreP:0.7969
Episode:82 meanR:28.0000 R:28.0000 rate:0.0560 aloss:0.6960 eloss:1.9340 aloss2:1.4921 exploreP:0.7947
Episode:83 meanR:27.9167 R:21.0000 rate:0.0420 aloss:0.6843 eloss:1.9168 aloss2:1.5005 exploreP:0.7931
Episode:84 meanR:28.2706 R:58.0000 rate:0.1160 aloss:0.6935 eloss:1.8983 aloss2:1.5346 exploreP:0.7885
Episode:85 meanR:28.1279 R:16.0000 rate:0.0320 aloss:0.7026 eloss:1.8788 aloss2:1.5576 exploreP:0.7873
Episode:86 meanR:28.6667 R:75.0000 rate:0.1500 aloss:0.7068 eloss:1.8406 aloss2:1.5994 exploreP:0.7815
Episode:87 meanR:28.4886 R:13.0000 rate:0.0260 aloss:0.6991 eloss:1.8133 aloss2:1.6479 exploreP:0.7805
Episode:88 meanR:28.3034 R:12.0000 rate:0.0240 aloss:0.7091 eloss:1.8175 aloss2:1.6487 exploreP:0.7795
Episode:89 meanR:28.3222 R:30.0000 rate:0.0600 aloss:0.7138 eloss:1.7881 

Episode:160 meanR:30.7400 R:136.0000 rate:0.2720 aloss:0.6777 eloss:1.7308 aloss2:1.5780 exploreP:0.6249
Episode:161 meanR:31.3100 R:78.0000 rate:0.1560 aloss:0.6788 eloss:1.7222 aloss2:1.5859 exploreP:0.6201
Episode:162 meanR:31.7700 R:67.0000 rate:0.1340 aloss:0.6783 eloss:1.7155 aloss2:1.6007 exploreP:0.6160
Episode:163 meanR:31.5700 R:14.0000 rate:0.0280 aloss:0.6717 eloss:1.7078 aloss2:1.6088 exploreP:0.6152
Episode:164 meanR:31.2900 R:29.0000 rate:0.0580 aloss:0.6782 eloss:1.7098 aloss2:1.6115 exploreP:0.6134
Episode:165 meanR:30.9200 R:37.0000 rate:0.0740 aloss:0.6761 eloss:1.7053 aloss2:1.6287 exploreP:0.6112
Episode:166 meanR:31.2000 R:44.0000 rate:0.0880 aloss:0.6884 eloss:1.6907 aloss2:1.6401 exploreP:0.6085
Episode:167 meanR:31.0300 R:17.0000 rate:0.0340 aloss:0.6865 eloss:1.6936 aloss2:1.6466 exploreP:0.6075
Episode:168 meanR:30.7700 R:11.0000 rate:0.0220 aloss:0.6734 eloss:1.6828 aloss2:1.6598 exploreP:0.6069
Episode:169 meanR:30.7400 R:33.0000 rate:0.0660 aloss:0.6787 el

Episode:239 meanR:57.9300 R:53.0000 rate:0.1060 aloss:0.7051 eloss:1.3581 aloss2:2.5699 exploreP:0.3951
Episode:240 meanR:58.0100 R:47.0000 rate:0.0940 aloss:0.6967 eloss:1.3582 aloss2:2.5721 exploreP:0.3933
Episode:241 meanR:58.0200 R:68.0000 rate:0.1360 aloss:0.7070 eloss:1.3579 aloss2:2.5766 exploreP:0.3907
Episode:242 meanR:58.4100 R:90.0000 rate:0.1800 aloss:0.7090 eloss:1.3526 aloss2:2.5817 exploreP:0.3872
Episode:243 meanR:58.3500 R:42.0000 rate:0.0840 aloss:0.7000 eloss:1.3566 aloss2:2.5882 exploreP:0.3857
Episode:244 meanR:59.2600 R:121.0000 rate:0.2420 aloss:0.6889 eloss:1.3641 aloss2:2.6006 exploreP:0.3811
Episode:245 meanR:58.8500 R:37.0000 rate:0.0740 aloss:0.6901 eloss:1.3458 aloss2:2.6067 exploreP:0.3798
Episode:246 meanR:59.4400 R:73.0000 rate:0.1460 aloss:0.6879 eloss:1.3676 aloss2:2.6139 exploreP:0.3771
Episode:247 meanR:59.6900 R:59.0000 rate:0.1180 aloss:0.6888 eloss:1.3525 aloss2:2.6080 exploreP:0.3749
Episode:248 meanR:59.3700 R:33.0000 rate:0.0660 aloss:0.6939 el

Episode:318 meanR:95.3200 R:116.0000 rate:0.2320 aloss:0.6659 eloss:1.2440 aloss2:2.9263 exploreP:0.1844
Episode:319 meanR:96.2500 R:133.0000 rate:0.2660 aloss:0.6688 eloss:1.2479 aloss2:2.9331 exploreP:0.1821
Episode:320 meanR:95.9300 R:88.0000 rate:0.1760 aloss:0.6747 eloss:1.2491 aloss2:2.9300 exploreP:0.1806
Episode:321 meanR:95.5900 R:50.0000 rate:0.1000 aloss:0.6717 eloss:1.2100 aloss2:2.9433 exploreP:0.1797
Episode:322 meanR:96.2400 R:102.0000 rate:0.2040 aloss:0.6756 eloss:1.2176 aloss2:2.9559 exploreP:0.1780
Episode:323 meanR:95.6000 R:93.0000 rate:0.1860 aloss:0.6706 eloss:1.2341 aloss2:2.9677 exploreP:0.1765
Episode:324 meanR:95.2400 R:60.0000 rate:0.1200 aloss:0.6653 eloss:1.2786 aloss2:2.9716 exploreP:0.1755
Episode:325 meanR:94.7900 R:53.0000 rate:0.1060 aloss:0.6601 eloss:1.2677 aloss2:2.9683 exploreP:0.1746
Episode:326 meanR:95.2500 R:117.0000 rate:0.2340 aloss:0.6690 eloss:1.2221 aloss2:2.9784 exploreP:0.1727
Episode:327 meanR:94.6800 R:66.0000 rate:0.1320 aloss:0.6642

...

Episode:5986 meanR:115.6500 R:10.0000 rate:0.0200 aloss:0.4475 eloss:1.4491 aloss2:14.0376 exploreP:0.0100
Episode:5987 meanR:115.9900 R:315.0000 rate:0.6300 aloss:0.4515 eloss:1.5183 aloss2:10.2468 exploreP:0.0100
Episode:5988 meanR:117.0100 R:500.0000 rate:1.0000 aloss:0.4682 eloss:1.4923 aloss2:10.6162 exploreP:0.0100
Episode:5989 meanR:116.8500 R:10.0000 rate:0.0200 aloss:0.4712 eloss:1.6164 aloss2:15.3712 exploreP:0.0100
Episode:5990 meanR:116.8300 R:136.0000 rate:0.2720 aloss:0.4697 eloss:1.4953 aloss2:9.7227 exploreP:0.0100
Episode:5991 meanR:121.6300 R:500.0000 rate:1.0000 aloss:0.4736 eloss:1.5102 aloss2:10.2277 exploreP:0.0100
Episode:5992 meanR:119.9700 R:183.0000 rate:0.3660 aloss:0.4656 eloss:1.4977 aloss2:9.4216 exploreP:0.0100
Episode:5993 meanR:119.0400 R:93.0000 rate:0.1860 aloss:0.4635 eloss:1.5799 aloss2:8.4874 exploreP:0.0100
Episode:5994 meanR:123.7700 R:500.0000 rate:1.0000 aloss:0.4672 eloss:1.5613 aloss2:10.8398 exploreP:0.0100
Episode:5995 meanR:126.3300 R:283.

Episode:6063 meanR:368.2200 R:500.0000 rate:1.0000 aloss:0.4853 eloss:1.7180 aloss2:5.2473 exploreP:0.0100
Episode:6064 meanR:372.8700 R:500.0000 rate:1.0000 aloss:0.4820 eloss:1.7238 aloss2:5.1810 exploreP:0.0100
Episode:6065 meanR:376.1200 R:500.0000 rate:1.0000 aloss:0.4681 eloss:1.7364 aloss2:4.7797 exploreP:0.0100
Episode:6066 meanR:380.3700 R:500.0000 rate:1.0000 aloss:0.5104 eloss:1.7253 aloss2:5.8977 exploreP:0.0100
Episode:6067 meanR:382.7400 R:500.0000 rate:1.0000 aloss:0.4678 eloss:1.7399 aloss2:4.5833 exploreP:0.0100
Episode:6068 meanR:383.5000 R:500.0000 rate:1.0000 aloss:0.5117 eloss:1.7253 aloss2:5.6394 exploreP:0.0100
Episode:6069 meanR:387.4400 R:500.0000 rate:1.0000 aloss:0.5066 eloss:1.7279 aloss2:5.3735 exploreP:0.0100
Episode:6070 meanR:390.7600 R:500.0000 rate:1.0000 aloss:0.4846 eloss:1.7151 aloss2:4.9407 exploreP:0.0100
Episode:6071 meanR:394.0100 R:500.0000 rate:1.0000 aloss:0.4709 eloss:1.7278 aloss2:4.4101 exploreP:0.0100
Episode:6072 meanR:398.7100 R:500.000

Episode:6140 meanR:473.2900 R:500.0000 rate:1.0000 aloss:0.4919 eloss:1.7893 aloss2:3.1370 exploreP:0.0100
Episode:6141 meanR:473.2900 R:500.0000 rate:1.0000 aloss:0.4885 eloss:1.7971 aloss2:3.0058 exploreP:0.0100
Episode:6142 meanR:473.2900 R:500.0000 rate:1.0000 aloss:0.4890 eloss:1.8086 aloss2:3.0075 exploreP:0.0100
Episode:6143 meanR:469.3000 R:101.0000 rate:0.2020 aloss:0.4754 eloss:1.8085 aloss2:2.6719 exploreP:0.0100
Episode:6144 meanR:469.3000 R:500.0000 rate:1.0000 aloss:0.4843 eloss:1.8108 aloss2:2.8590 exploreP:0.0100
Episode:6145 meanR:469.3000 R:500.0000 rate:1.0000 aloss:0.4873 eloss:1.8193 aloss2:2.8120 exploreP:0.0100
Episode:6146 meanR:469.8600 R:403.0000 rate:0.8060 aloss:0.4729 eloss:1.8063 aloss2:2.6318 exploreP:0.0100
Episode:6147 meanR:469.8600 R:500.0000 rate:1.0000 aloss:0.4816 eloss:1.8403 aloss2:2.6595 exploreP:0.0100
Episode:6148 meanR:469.8600 R:500.0000 rate:1.0000 aloss:0.4918 eloss:1.8363 aloss2:2.8592 exploreP:0.0100
Episode:6149 meanR:469.8600 R:500.000

Episode:6217 meanR:470.0000 R:500.0000 rate:1.0000 aloss:0.4197 eloss:1.5971 aloss2:2.7029 exploreP:0.0100
Episode:6218 meanR:470.0000 R:500.0000 rate:1.0000 aloss:0.4181 eloss:1.5943 aloss2:2.6952 exploreP:0.0100
Episode:6219 meanR:470.0000 R:500.0000 rate:1.0000 aloss:0.4177 eloss:1.5906 aloss2:2.7136 exploreP:0.0100
Episode:6220 meanR:465.1000 R:10.0000 rate:0.0200 aloss:0.4155 eloss:1.5653 aloss2:2.8028 exploreP:0.0100
Episode:6221 meanR:468.0400 R:435.0000 rate:0.8700 aloss:0.4152 eloss:1.5778 aloss2:2.7654 exploreP:0.0100
Episode:6222 meanR:464.7700 R:173.0000 rate:0.3460 aloss:0.4156 eloss:1.5792 aloss2:2.7561 exploreP:0.0100
Episode:6223 meanR:464.7700 R:500.0000 rate:1.0000 aloss:0.4165 eloss:1.5955 aloss2:2.7690 exploreP:0.0100
Episode:6224 meanR:468.6100 R:500.0000 rate:1.0000 aloss:0.4139 eloss:1.5701 aloss2:2.7774 exploreP:0.0100
Episode:6225 meanR:468.6100 R:500.0000 rate:1.0000 aloss:0.4138 eloss:1.5719 aloss2:2.7565 exploreP:0.0100
Episode:6226 meanR:463.7100 R:10.0000 

Episode:6294 meanR:305.9200 R:18.0000 rate:0.0360 aloss:0.3759 eloss:1.3124 aloss2:3.7290 exploreP:0.0100
Episode:6295 meanR:301.1000 R:18.0000 rate:0.0360 aloss:0.3661 eloss:1.3369 aloss2:3.6589 exploreP:0.0100
Episode:6296 meanR:299.5300 R:343.0000 rate:0.6860 aloss:0.3708 eloss:1.3149 aloss2:3.7071 exploreP:0.0100


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
